# Human-in-the-loop Breakpoints

- streams & user input injection

## Approval

User case.

what we need:

- a way to stop the execution
- a way to resume the execution

## Redefine financial advicer graph (from 03_agent.ipynb)

In [ ]:
import requests
import yfinance as yf
from pprint import pformat
from langchain_core.tools import Tool
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import MessagesState, START, StateGraph
from langgraph.prebuilt import tools_condition, ToolNode
from langgraph.checkpoint.memory import MemorySaver
from IPython.display import Image, display


# Defining Tools
##################################################################################

def lookup_stock_symbol(company_name: str) -> str:
    """
    Converts a company name to its stock symbol using a financial API.

    Parameters:
        company_name (str): The full company name (e.g., 'Tesla').

    Returns:
        str: The stock symbol (e.g., 'TSLA') or an error message.
    """
    api_url = "https://www.alphavantage.co/query"
    params = {
        "function": "SYMBOL_SEARCH",
        "keywords": company_name,
        "apikey": "your_alphavantage_api_key"
    }
    
    response = requests.get(api_url, params=params)
    data = response.json()
    
    if "bestMatches" in data and data["bestMatches"]:
        return data["bestMatches"][0]["1. symbol"]
    else:
        return f"Symbol not found for {company_name}."


def fetch_stock_data_raw(stock_symbol: str) -> dict:
    """
    Fetches comprehensive stock data for a given symbol and returns it as a combined dictionary.

    Parameters:
        stock_symbol (str): The stock ticker symbol (e.g., 'TSLA').
        period (str): The period to analyze (e.g., '1mo', '3mo', '1y').

    Returns:
        dict: A dictionary combining general stock info and historical market data.
    """
    period = "1mo"
    try:
        stock = yf.Ticker(stock_symbol)

        # Retrieve general stock info and historical market data
        stock_info = stock.info  # Basic company and stock data
        stock_history = stock.history(period=period).to_dict()  # Historical OHLCV data

        # Combine both into a single dictionary
        combined_data = {
            "stock_symbol": stock_symbol,
            "info": stock_info,
            "history": stock_history
        }

        return pformat(combined_data)

    except Exception as e:
        return {"error": f"Error fetching stock data for {stock_symbol}: {str(e)}"}


# Binding tools to the LLM
##################################################################################

# Create tool bindings with additional attributes
lookup_stock = Tool.from_function(
    func=lookup_stock_symbol,
    name="lookup_stock_symbol",
    description="Converts a company name to its stock symbol using a financial API.",
    return_direct=False  # Return result to be processed by LLM
)

fetch_stock = Tool.from_function(
    func=fetch_stock_data_raw,
    name="fetch_stock_data_raw",
    description="Fetches comprehensive stock data including general info and historical market data for a given stock symbol.",
    return_direct=False
)

toolbox = [lookup_stock, fetch_stock]

# OPENAI_API_KEY environment variable must be set
simple_llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = simple_llm.bind_tools(toolbox)


# Defining Agent's node
##################################################################################

# System message
assistant_system_message = SystemMessage(content=("""
You are a professional financial assistant specializing in stock market analysis and investment strategies. 
Your role is to analyze stock data and provide **clear, decisive recommendations** that users can act on, 
whether they already hold the stock or are considering investing.

You have access to a set of tools that can provide the data you need to analyze stocks effectively. 
Use these tools to gather relevant information such as stock symbols, current prices, historical trends, 
and key financial indicators. Your goal is to leverage these resources efficiently to generate accurate, 
actionable insights for the user.

Your responses should be:
- **Concise and direct**, summarizing only the most critical insights.
- **Actionable**, offering clear guidance on whether to buy, sell, hold, or wait for better opportunities.
- **Context-aware**, considering both current holders and potential investors.
- **Free of speculation**, relying solely on factual data and trends.

### Response Format:
1. **Recommendation:** Buy, Sell, Hold, or Wait.
2. **Key Insights:** Highlight critical trends and market factors that influence the decision.
3. **Suggested Next Steps:** What the user should do based on their current position.

If the user does not specify whether they own the stock, provide recommendations for both potential buyers and current holders. Ensure your advice considers valuation, trends, and market sentiment.

Your goal is to help users make informed financial decisions quickly and confidently.
"""))

# Node
def assistant(state: MessagesState):
   return {"messages": [llm_with_tools.invoke([assistant_system_message] + state["messages"])]}


# Defining Graph
##################################################################################
# Graph
builder = StateGraph(MessagesState)

# Define nodes: these do the work
builder.add_node("assistant", assistant)
builder.add_node("tools", ToolNode(toolbox))

# Define edges: these determine how the control flow moves
builder.add_edge(START, "assistant")
builder.add_conditional_edges(
    "assistant",
    # If the latest message (result) from assistant is a tool call -> tools_condition routes to tools
    # If the latest message (result) from assistant is a not a tool call -> tools_condition routes to END
    tools_condition,
)
builder.add_edge("tools", "assistant")

memory = MemorySaver()
graph = builder.compile(checkpointer=memory)
display(Image(graph.get_graph().draw_mermaid_png()))

## Adding a breakpoint to the graph

In [ ]:
graph = builder.compile(interrupt_before=["tools"], checkpointer=memory)

display(Image(graph.get_graph(xray=True).draw_mermaid_png()))

## Let's test the breakpoint

In [ ]:
# start a new conversation
thread = {"configurable": {"thread_id": "1"}}

# define intiial user request
initial_input = {"messages": HumanMessage(content="Should I invest in Tesla stocks?")}

# run the graph and stream in values mode
for event in graph.stream(initial_input, thread, stream_mode="values"):
    event['messages'][-1].pretty_print()

## Check the graph current state

In [ ]:
state = graph.get_state(thread)
state

## Get the next node in the graph

In [ ]:
state.next

## Resuming execution

### 1) Initializes a new execution flow based on the provided initial_input

```python
graph.stream(initial_input, thread, stream_mode="values")
```

### 2) resumes from the latest checkpoint stored in the thread state

```python
graph.stream(None, thread, stream_mode="values")
```

Passing `None` as initial_input tells LangGraph: “Continue execution from where it left off for the given `thread_id`.”

By setting initial_input = `None`, you’re not reinitializing the graph but rather instructing it to continue from the last recorded execution point for the given `thread_id`. This mechanism ensures smooth, stateful interactions, making cyclic approvals and multi-step workflows possible.

In [ ]:
# as a first event LangGraph re-emits the current state (AIMessage with the tool call).
for event in graph.stream(None, thread, stream_mode="values"):
    event['messages'][-1].pretty_print()

## Combining all together

In [ ]:
# Start a new conversation
thread = {"configurable": {"thread_id": "3"}}

initial_input = {"messages": HumanMessage(content="Should I invest in Tesla stocks?")}

current_state_reemit = False

# Run the graph in a loop, pausing at each tool-node call
while True:
    should_restart = False  # Reset flag at the start of each while-loop iteration

    # Run the graph until the next interruption (breakpoint before tool-node)
    for event in graph.stream(initial_input, thread, stream_mode="values"):
        event['messages'][-1].pretty_print()
        
        # Check if a tool is about to be called
        if "tool_calls" in event['messages'][-1].additional_kwargs:
            if  current_state_reemit:
                current_state_reemit = False
                continue

            else:
                tool_name = event['messages'][-1].additional_kwargs["tool_calls"][0]["function"]["name"]  # Extract the tool name
                # Get user feedback
                user_approval = input(f"Do you want to call the tool {tool_name}? (yes/no): ")

                if user_approval.lower() == "yes":
                    initial_input = None
                    current_state_reemit = True
                    should_restart = True
                else:
                    print(f"\n\nTool call {tool_name} was cancelled by user")

                break

    if should_restart:
        continue
    
    # Once the graph completes without hitting a tool node, break out of the loop
    break

## Breakpoints in LangGraph API

In [ ]:
from langgraph_sdk import get_client

URL = "http://localhost:49381"
client = get_client(url=URL)

assistants = await client.assistants.search()
assistants

In [ ]:
from langchain_core.messages import convert_to_messages

thread = await client.threads.create()
input_message = HumanMessage(content="Should I invest in Tesla stocks?")

async for event in client.runs.stream(
            thread["thread_id"], 
            assistant_id="b7480eb0-6390-53a5-9bc4-29bf27cbd1c4", 
            input={"messages": [input_message]}, 
            stream_mode="values",
            interrupt_before=["tools"],
):
    messages = event.data.get('messages', None)
    if messages:
        print(convert_to_messages(messages)[-1])

In [ ]:
async for event in client.runs.stream(
            thread["thread_id"], 
            assistant_id="b7480eb0-6390-53a5-9bc4-29bf27cbd1c4", 
            input=None, 
            stream_mode="values",
            interrupt_before=["tools"],
):
    messages = event.data.get('messages', None)
    if messages:
        print(convert_to_messages(messages)[-1])

In [ ]:
async for event in client.runs.stream(
            thread["thread_id"], 
            assistant_id="b7480eb0-6390-53a5-9bc4-29bf27cbd1c4", 
            input=None, 
            stream_mode="values",
            interrupt_before=["tools"],
):
    messages = event.data.get('messages', None)
    if messages:
        print(convert_to_messages(messages)[-1])

## Breakpoints in Langgraph Studio

Let's copy/paste `financial_advisor` but with `interrupt_before=["tools"]`. (`studio/financial_advisor.py` -> `studio/financial_advisor_breakpoint.py`)